In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

client_credentials_manager = SpotifyClientCredentials(
        "26399552a8ce4d1285397254189cac50",
        "fdacfffa2dd34dbeb127dedb459f7ea3")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
def getTrackFeatures(trackURI):
    ger_data = pd.DataFrame()
    for track in trackURI:
        audio = sp.audio_analysis(track)
        segments_data =  pd.DataFrame(audio["segments"])
        pitch = segments_data.pitches.apply(pd.Series)
        pitch.columns = ["p" + str(i) for i in range(1,13)]
        timbre = segments_data.timbre.apply(pd.Series)
        timbre.columns = ["t" + str(i) for i in range(1,13)]
        segments_data = segments_data.drop(["pitches", "timbre", "loudness_end"],
                                           axis = 1)
        segments_data = segments_data.join([pitch, timbre])
        segments_data["track_id"] = track
        ger_data = ger_data.append(segments_data)
    return ger_data

In [ ]:
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Oceania
print(len(countriesOfInterest))
for year in range(2019,2000, -1):
    tmstamp = str(year) + "-06-15T12:00:00"
    for countryCode in countriesOfInterest:
        country = sp.featured_playlists(country = countryCode, limit = 50)
        plist = country["playlists"]["items"]
        print(countryCode, "has", len(plist), "playlists")
        plist = pd.DataFrame.from_dict(plist)
        plist.to_csv("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv", index = False)
        for uri in plist["uri"]:
            currplist = sp.playlist(uri)
            trackURI = []
            for track in currplist["tracks"]["items"]:
                try:
                    trackURI = trackURI + [track["track"]["uri"]]
                except:
                    continue
            print("Getting track features for", uri, "in", countryCode)
            trackData = getTrackFeatures(trackURI)
            trackData.to_csv("Raw Track Data\\" + countryCode + "_" + str(year) + "_" + uri[17:] + ".csv", index = False)

27
DZ has 16 playlists
Getting track features for spotify:playlist:37i9dQZF1DWZKucvJjqs4a in DZ


In [ ]:
for year in range(2019,2000, -1):
    print(year)